<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Groupby and Joins
              
</p>
</div>

Data Science Cohort Live NYC Feb 2022
<p>Phase 1: Topic 4</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

In [9]:
import pandas as pd
import sqlite3
import pandas as pd
conn = sqlite3.connect("data/flights.db")
cur = conn.cursor() 

We've seen aggregations last lecture:

In [10]:
# Max value for longitude
pd.read_sql('''
    SELECT 
    
         name, MAX(
            CAST(longitude AS REAL) 
        ) AS max_long
    FROM 
        airports
''', conn)

name  max_long
0  Moala Airport   179.951

In [11]:
%%bash

sqlite3 data/flights.db
.schema airlines

CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
);
CREATE INDEX ix_airlines_index ON airlines ([index]);


In [12]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT(*) AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

number_of_active_airlines
0                       1161

This is OK, but want to:
- often compute aggregations across different groups 

#### Grouping in SQL

- Group by values of a categorical column
- Apply aggregation/transformation to groups

GROUP BY statement -- typically used with aggregation!



SELECT colwithgroups,<br>
       agg_function(another_col)<br>
FROM table<br>
GROUP BY colwithgroups<br>

In [13]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines
        LIMIT 5
''', conn)
df_results

index id                                          name alias  iata  icao  \
0      0  1                                Private flight    \N     -  None   
1      1  2                                   135 Airways    \N  None   GNL   
2      2  3                                 1Time Airline    \N    1T   RNX   
3      3  4  2 Sqn No 1 Elementary Flying Training School    \N  None   WYT   
4      4  5                               213 Flight Unit    \N  None   TFU   

  callsign         country active  
0     None            None      Y  
1  GENERAL   United States      N  
2  NEXTIME    South Africa      Y  
3     None  United Kingdom      N  
4     None          Russia      N

In [25]:
df_results = pd.read_sql('''
    SELECT 
        country, COUNT(*) AS number_of_airlines
    FROM 
        airlines
    WHERE
        active='Y'
    GROUP BY
        country
''', conn)

df_results

country  number_of_airlines
0           None                   2
1           S.A.                   1
2         ALASKA                   1
3        AVIANCA                   1
4    Afghanistan                   4
..           ...                 ...
190      Vietnam                   7
191        Yemen                   2
192       Zambia                   3
193     Zimbabwe                   3
194           \N                   2

[195 rows x 2 columns]

Let's see another example finding the airport with the highest altitude in each country:

In [24]:
df_results = pd.read_sql('''
    
''', conn)

df_results

DatabaseError: Execution failed on sql '
    SELECT 
        country,
        name as airport_name, MAX(CAST(altitude AS int)) AS altitude
    FROM 
        airports
    GROUP BY
        country
    WHERE
        active = 'Y'
': near "WHERE": syntax error

Let's look at the head of the airlines table:

In [16]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines 
    LIMIT 3
''', conn)

df_results

index id            name alias  iata  icao callsign        country active
0      0  1  Private flight    \N     -  None     None           None      Y
1      1  2     135 Airways    \N  None   GNL  GENERAL  United States      N
2      2  3   1Time Airline    \N    1T   RNX  NEXTIME   South Africa      Y

Query challenge: Count the number of active flights grouped by country.

In [22]:
# let's do it!
df_results = pd.read_sql('''
    SELECT
        country, active, COUNT(*) AS number_of_airlines
    FROM
        airlines
    GROUP BY
        country

''', conn)

df_results

country active  number_of_airlines
0                     None      Y                  15
1     Boonville Stage Line      N                   1
2                     S.A.      Y                   1
3                     ACOM      N                   1
4              ACTIVE AERO      N                   1
..                     ...    ...                 ...
272               WATCHDOG      N                   1
273                  Yemen      N                   3
274                 Zambia      N                  23
275               Zimbabwe      Y                   7
276                     \N      Y                   3

[277 rows x 3 columns]

**Exercise**: let's get fancy pantsy 

- Combining CASE-WHEN and groupbys

- Partition the airports into three latitude regions: 

- northern (>= 30), central (30,-30), and southern zones (<=-30).

Get me the airports in each zone with the maximum altitude

In [38]:
# let's do it!
df_results = pd.read_sql('''

    SELECT name AS airport_name, country, MAX(CAST(altitude AS float)) AS max_alt,
    CASE 
        WHEN CAST(altitude AS FLOAT) >= 30 THEN "northern"
        WHEN CAST(altitude AS FLOAT) <= -30 THEN "southern"
        ELSE "Central"
        
    END AS lat_category
    
    FROM airports
    
    GROUP BY lat_category

    
''', conn)

df_results

airport_name         country  max_alt lat_category
0    Church Fenton  United Kingdom     29.0      Central
1  Yading Daocheng           China  14472.0     northern
2            Rasht            Iran    -40.0     southern

#### Filtering Groups with HAVING

- Can filter columns/tables with WHERE
- WHERE doesn't work when filtering aggregations on groupby
- `HAVING` will do the trick. 

Let's only return countries with active airlines having a count greater than 30:

In [46]:
pd.read_sql('''
    SELECT 
        country,
        COUNT(*) AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING
        num > 30
    ORDER BY 
        num DESC
''', conn)

country  num
0   United States  141
1          Russia   72
2  United Kingdom   40
3         Germany   37
4          Canada   34

- Can use multiple aggregation functions in groupby.
- Filter on one of the aggregations.

Get the altitude of the highest airport in a country given that it has at least 100 airports.

In [40]:
pd.read_sql('''
    SELECT 
        country,
        COUNT(*) AS num_airports,
        MAX(CAST(altitude as int)) as max_altitude
    FROM 
        airports
    GROUP BY 
        country
    HAVING
        num_airports >= 100
''', conn)

country  num_airports  max_altitude
0        Argentina           103         11414
1        Australia           263          4260
2           Brazil           213          8708
3           Canada           435          3939
4            China           219         14472
5           France           233          6588
6          Germany           321          2297
7            India           140         10682
8        Indonesia           114          9104
9            Japan           131          2200
10          Russia           249         13411
11    South Africa           103          5700
12  United Kingdom           210           811
13   United States          1697          9308

**Exercise**

Get the number of airports of cities having more than one airport and the average airport altitude in the city.

Use GROUP BY and HAVING clauses for this.

In [46]:
pd.read_sql('''
   SELECT 
        city,
        COUNT(*) AS num_airports,
        AVG(CAST(altitude as float)) as average_altitude
    FROM 
        airports
    GROUP BY 
        city
    HAVING
        num_airports > 1

''', conn)

city  num_airports  average_altitude
0          Aachen             2        721.500000
1        Aberdeen             5        315.000000
2         Abilene             2       1790.000000
3       Abu Dhabi             3         60.333333
4           Adana             3        267.666667
..            ...           ...               ...
661     Yuryevets             2        134.500000
662        Zagreb             2        426.500000
663        Zhuhai             2          1.000000
664        Zurich             2       1377.500000
665  Zweibruecken             2       1132.500000

[666 rows x 3 columns]

#### Joins
- Saw this in Pandas
- Join information from two tables in a relational database together.
- For the two tables: can relate via a common key (aka foreign key).

SQL has many different kinds of joins. We only look at two:
- INNER JOIN
- LEFT JOIN 

A little picture:

<div>
<center><img src="Images/venn.png" align = "center" width="600"/></center>
</div>

#### Inner joins

- Join two tables together on a shared key. 
- Keep rows if the key is in both tables.


SELECT
    table1.column_name, <br>
    table2.different_column_name <br>
FROM <br>
    table1 <br>
    INNER JOIN table2 <br>
        ON table1.shared_column_name = table2.shared_column_name <br>

##### Syntax: Inner Join
SELECT
    table1.column_name, <br>
    table2.different_column_name <br>
FROM <br>
    table1 <br>
    INNER JOIN table2 <br>
        ON table1.shared_column_name = table2.shared_column_name <br>

Let's check out the airports and airline table. 

In [10]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes
    LIMIT 5
    

''', conn)

index airline airline_id source source_id dest dest_id codeshare stops  \
0      0      2B        410    AER      2965  KZN    2990      None     0   
1      1      2B        410    ASF      2966  KZN    2990      None     0   
2      2      2B        410    ASF      2966  MRV    2962      None     0   
3      3      2B        410    CEK      2968  KZN    2990      None     0   
4      4      2B        410    CEK      2968  OVB    4078      None     0   

  equipment  
0       CR2  
1       CR2  
2       CR2  
3       CR2  
4       CR2

In [49]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines
        LIMIT 5

''', conn)

index id                                          name alias  iata  icao  \
0      0  1                                Private flight    \N     -  None   
1      1  2                                   135 Airways    \N  None   GNL   
2      2  3                                 1Time Airline    \N    1T   RNX   
3      3  4  2 Sqn No 1 Elementary Flying Training School    \N  None   WYT   
4      4  5                               213 Flight Unit    \N  None   TFU   

  callsign         country active  
0     None            None      Y  
1  GENERAL   United States      N  
2  NEXTIME    South Africa      Y  
3     None  United Kingdom      N  
4     None          Russia      N

In [51]:
pd.read_sql('''
    SELECT 
        al.name, al.callsign, rt.airline_id, rt.source, rt.dest
    FROM
        airlines AS al
        INNER JOIN
        routes AS rt
        ON al.id = rt.airline_id
        

''', conn)

name     callsign airline_id source dest
0                40-Mile Air     MILE-AIR         10    CKX  TKJ
1                40-Mile Air     MILE-AIR         10    FAI  HKB
2                40-Mile Air     MILE-AIR         10    HKB  FAI
3                40-Mile Air     MILE-AIR         10    TKJ  CKX
4                 Aigle Azur   AIGLE AZUR         21    AAE  MRS
...                      ...          ...        ...    ...  ...
66980  Rainbow Air Polynesia  Rainbow Air      19676    DAC  ZYL
66981  Rainbow Air Polynesia  Rainbow Air      19676    JSR  DAC
66982  Rainbow Air Polynesia  Rainbow Air      19676    KUL  DAC
66983  Rainbow Air Polynesia  Rainbow Air      19676    SIN  DAC
66984  Rainbow Air Polynesia  Rainbow Air      19676    ZYL  DAC

[66985 rows x 5 columns]

The airline_id in the routes table and id column in airlines are common keys.

Can join these two tables keeping rows with keys in both tables.

- Note table aliasing

#### LEFT JOIN 

Can join two tables keeping rows with keys in left table only.

In [53]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines as al
        
    LEFT JOIN routes as rt
            ON al.id = rt.airline_id
''', conn)

index     id                                          name  \
0          0      1                                Private flight   
1          1      2                                   135 Airways   
2          2      3                                 1Time Airline   
3          3      4  2 Sqn No 1 Elementary Flying Training School   
4          4      5                               213 Flight Unit   
...      ...    ...                                           ...   
72488   6043  19828                                    Vuela Cuba   
72489   6044  19830                                 All Australia   
72490   6045  19831                                    Fly Europa   
72491   6046  19834                                   FlyPortugal   
72492   6047  19845                          FTI Fluggesellschaft   

               alias  iata  icao     callsign         country active  index  \
0                 \N     -  None         None            None      Y    NaN   
1                 \N  None   GNL      GENERAL   United States      N    NaN   
2                 \N    1T   RNX      NEXTIME    South Africa      Y    NaN   
3                 \N  None   WYT         None  United Kingdom      N    NaN   
4                 \N  None   TFU         None          Russia      N    NaN   
...              ...   ...   ...          ...             ...    ...    ...   
72488     Vuela Cuba    6C   6CC         None            Cuba      Y    NaN   
72489  All Australia    88   8K8         None       Australia      Y    NaN   
72490           None    ER   RWW         None           Spain      Y    NaN   
72491           None    PO   FPT  FlyPortugal        Portugal      Y    NaN   
72492           None  None   FTI         None         Germany      N    NaN   

      airline airline_id source source_id  dest dest_id codeshare stops  \
0        None       None   None      None  None    None      None  None   
1        None       None   None      None  None    None      None  None   
2        None       None   None      None  None    None      None  None   
3        None       None   None      None  None    None      None  None   
4        None       None   None      None  None    None      None  None   
...       ...        ...    ...       ...   ...     ...       ...   ...   
72488    None       None   None      None  None    None      None  None   
72489    None       None   None      None  None    None      None  None   
72490    None       None   None      None  None    None      None  None   
72491    None       None   None      None  None    None      None  None   
72492    None       None   None      None  None    None      None  None   

      equipment  
0          None  
1          None  
2          None  
3          None  
4          None  
...         ...  
72488      None  
72489      None  
72490      None  
72491      None  
72492      None  

[72493 rows x 19 columns]

Can do it the other way:

In [14]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes as rt
        
    LEFT JOIN airlines as al
            ON rt.airline_id = al.id 
''', conn)

index airline airline_id source source_id dest dest_id codeshare stops  \
0          0      2B        410    AER      2965  KZN    2990      None     0   
1          1      2B        410    ASF      2966  KZN    2990      None     0   
2          2      2B        410    ASF      2966  MRV    2962      None     0   
3          3      2B        410    CEK      2968  KZN    2990      None     0   
4          4      2B        410    CEK      2968  OVB    4078      None     0   
...      ...     ...        ...    ...       ...  ...     ...       ...   ...   
67658  67658      ZL       4178    WYA      6334  ADL    3341      None     0   
67659  67659      ZM      19016    DME      4029  FRU    2912      None     0   
67660  67660      ZM      19016    FRU      2912  DME    4029      None     0   
67661  67661      ZM      19016    FRU      2912  OSS    2913      None     0   
67662  67662      ZM      19016    OSS      2913  FRU    2912      None     0   

      equipment   index     id              name   alias iata icao  \
0           CR2   409.0    410        Aerocondor      \N   2B  ARD   
1           CR2   409.0    410        Aerocondor      \N   2B  ARD   
2           CR2   409.0    410        Aerocondor      \N   2B  ARD   
3           CR2   409.0    410        Aerocondor      \N   2B  ARD   
4           CR2   409.0    410        Aerocondor      \N   2B  ARD   
...         ...     ...    ...               ...     ...  ...  ...   
67658       SF3  4171.0   4178  Regional Express      \N   ZL  RXA   
67659       734  5977.0  19016        Apache Air  Apache   ZM  IWA   
67660       734  5977.0  19016        Apache Air  Apache   ZM  IWA   
67661       734  5977.0  19016        Apache Air  Apache   ZM  IWA   
67662       734  5977.0  19016        Apache Air  Apache   ZM  IWA   

         callsign        country active  
0      AEROCONDOR       Portugal      Y  
1      AEROCONDOR       Portugal      Y  
2      AEROCONDOR       Portugal      Y  
3      AEROCONDOR       Portugal      Y  
4      AEROCONDOR       Portugal      Y  
...           ...            ...    ...  
67658         REX      Australia      Y  
67659      APACHE  United States      Y  
67660      APACHE  United States      Y  
67661      APACHE  United States      Y  
67662      APACHE  United States      Y  

[67663 rows x 19 columns]

A lot more unique entries on shared key in the airline table vs. the route table.

We won't go through RIGHT JOIN and FULL JOIN here -- its actually fairly self explanatory what they do.

#### CROSS JOIN

This can be pretty useful when you want to:
- compare every row in one table with every other row in another table.
- create all possible combinations of columns belonging in different tables.
    

A little picture might help:

<div>
<center><img src="Images/cross_join.png" align = "center" width="600"/></center>
</div>

In [19]:
pd.read_sql('''
    SELECT rt.airline_id as rt_id,
           rt.source as rt_source,
           rt.dest as rt_dest,
           al.id as al_id,
           al.name as al_name
           
    FROM
        routes as rt
        CROSS JOIN airlines as al
        
    LIMIT 10
''', conn)

rt_id rt_source rt_dest al_id                                       al_name
0   410       AER     KZN     1                                Private flight
1   410       AER     KZN     2                                   135 Airways
2   410       AER     KZN     3                                 1Time Airline
3   410       AER     KZN     4  2 Sqn No 1 Elementary Flying Training School
4   410       AER     KZN     5                               213 Flight Unit
5   410       AER     KZN     6                 223 Flight Unit State Airline
6   410       AER     KZN     7                             224th Flight Unit
7   410       AER     KZN     8                                   247 Jet Ltd
8   410       AER     KZN     9                                   3D Aviation
9   410       AER     KZN    10                                   40-Mile Air

Be careful: cross joins can take a long time!

#### SELF JOIN

This is not exactly a standard join operation. But it can be very useful.

- Join a table with itself via inner join.

**Use case:** 

Create pairwise comparisons for airports in the same country and sharing the same city.

Here, we do an inner join:

- Can join on multiple columns and conditions!!
- Self join is excellent use case for this.

In [30]:
pd.read_sql('''
    SELECT ap1.country,
        ap1.city,
        ap1.name AS ap1_name, 
        ap1.code AS ap1_code,
        ap1.altitude as ap1_altitude,
        ap2.name AS ap2_name, 
        ap2.code AS ap2_code, 
        ap2.altitude as ap2_altitude
        
    FROM
        airports as ap1
        INNER JOIN airports  as ap2
            ON ap1.country = ap2.country
            AND ap1.city = ap2.city
            AND ap1.name <> ap2.name
''', conn)

country       city                           ap1_name ap1_code  \
0            Canada   Winnipeg                Winnipeg St Andrews      YAV   
1            Canada   Winnipeg                Winnipeg St Andrews      YAV   
2            Canada    Halifax                         Shearwater      YAW   
3            Canada    Nanaimo                            Nanaimo      YCD   
4            Canada    Chatham                          Miramichi      YCH   
...             ...        ...                                ...      ...   
3561  United States  San Diego  San Diego Old Town Transit Center      OLT   
3562  United States  San Diego  San Diego Old Town Transit Center      OLT   
3563  United States  San Diego  San Diego Old Town Transit Center      OLT   
3564  United States  San Diego  San Diego Old Town Transit Center      OLT   
3565  United States  San Diego  San Diego Old Town Transit Center      OLT   

     ap1_altitude                       ap2_name ap2_code ap2_altitude  
0             760                  Train Station      XEF          751  
1             760                  Winnipeg Intl      YWG          783  
2             167                   Halifax Intl      YHZ          477  
3              93  Nanaimo Harbour Water Airport      ZNA            0  
4             108                           Kent      XCM          645  
...           ...                            ...      ...          ...  
3561            0               Montgomery Field      MYF           17  
3562            0               North Island Nas      NZY           26  
3563            0                 San Diego Intl      SAN           17  
3564            0        San Diego Union Station     None           50  
3565            0        Sorrento Valley Station     None          150  

[3566 rows x 8 columns]

#### Set Operations

- UNION: gets union, drops duplicates
- UNION ALL: gets union, keeps duplicates
- INTERSECT: gets intersection
- EXCEPT: gets what's in one table but not other.

Useful when you have two tables with same data scheme.

- Table1 has some fields with same kind of data as some fields in Table2.
- Can take intersection of fields from two tables with same kind of data, etc.
- Less flexible than joins, but fast and useful in many cases.

<div>
<center><img src="Images/setopd.png" align = "center" width="600"/></center>
</div>

An example might be useful here:

Get countries in airports table not in airlines table.

Countries that have airports but no airlines.

In [17]:
pd.read_sql('''
    SELECT ap.country 
    FROM airports as ap
    EXCEPT
    SELECT al.country
    FROM airlines as al
    LIMIT 10
''', conn)

country
0                        Anguilla
1                      Antarctica
2  British Indian Ocean Territory
3                Christmas Island
4         Cocos (Keeling) Islands
5                        Dominica
6                      East Timor
7                Falkland Islands
8                       Gibraltar
9                       Greenland

Next up: subqueries!!

Can give us additional flexibility in accessing data!